In [ ]:
%matplotlib inline
import datetime
import numpy as np
import pathlib as pl
import pandas as pd
import geopandas as gpd
import sys
import xugrid

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
import matplotlib.dates as dates
import matplotlib.ticker as ticker

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import flopy
import flopy.plot.styles as styles

In [ ]:
sys.path.append("../common")
from liss_settings import (
                            cx, cx_provider, 
                            extent, boxx, boxy, 
                            mf6_model_crs, 
                            set_title_string,  
                            fig_ext, transparent, 
                            get_modflow_coupling_tag, 
                            get_dflow_dtuser, 
                            get_dflow_grid_name,
                          )

In [ ]:
figsize = (10.6, 5.95)
alpha = 0.5
basemap_kwds = {"attribution": False, "source": cx_provider, "zoom": 13}
progress_kwds = {"lw": 0.0, "fc": "cyan", "alpha": 0.25, "zorder": 10}
animation_ws = pl.Path("animations")
animation_ws.mkdir(exist_ok=True, parents=True)

#### Set unit conversion factors

In [ ]:
d2sec = 24. * 60. * 60.
hrs2sec = 60. * 60. 
m2ft = 3.28081
cfd2cms = 1.0 / ((3.28082**3) * 86400.)

In [ ]:
units = "mm"
conversion_factor = 1.0
if units == "mm":
    conversion_factor = 1000.0 / 3.28081
elif units == "in":
    conversion_factor = 12.0
head_units = "m"
head_conversion_factor = 1.0
if head_units == "m":
    head_conversion_factor = 1.0 / 3.28081
stage_units = head_units
stage_conversion_factor = 1.0
if stage_units == "ft":
    stage_conversion_factor = 3.28081
flux_units = units
if flux_units == "mm":
    flux_conversion_factor = 1000.0
elif flux_units == "in":
    flux_conversion_factor = 3.28081 * 12.


#### Set the MODFLOW coupling frequency

Change the `mf_couple_freq_hours` value. Only tested for multiple of the D-Flow FM DtUser variable. Will not work for `mf_couple_freq_hours` values greater than 24.

In [ ]:
control_path = pl.Path("../dflow-fm/coarse/tides/base/FlowFM.mdu") # change this if using a different D-Flow FM control file
grid_name = get_dflow_grid_name(control_path)
print(grid_name)

In [ ]:
coastal_dtuser = get_dflow_dtuser(control_path)
print(coastal_dtuser)

In [ ]:
mf_couple_freq_hours = 1.0  #Change this value to change the coupling frequency
mf_couple_freq = mf_couple_freq_hours * hrs2sec
coastal_per_mf = int(mf_couple_freq / coastal_dtuser)
mf_output_sample = int(24. / mf_couple_freq_hours)

In [ ]:
print(
    f"MODFLOW coupling frequency {mf_couple_freq_hours} hours\n" +
    f"MODFLOW coupled to the coastal every {coastal_per_mf} output time step ({coastal_dtuser} sec.)\n" +
    f"MODFLOW output sampling frequency {mf_output_sample}"
) 

In [ ]:
mf_tag = get_modflow_coupling_tag(mf_couple_freq_hours)
print(f"MODFLOW coupling tag: {mf_tag}")

#### Coastal model results (used for the calendar times)

In [ ]:
full_results_ds = xugrid.open_dataset("../dflow-fm/coarse/tides/run/output/FlowFM_map.nc")
full_results_ds

#### Extract subset of coastal model for plotting 

In [ ]:
results_ds = full_results_ds

In [ ]:
coastal_gdf = results_ds["mesh2d_nFaces"].ugrid.to_geodataframe(name="cell")
coastal_gdf["stage"] = np.zeros((results_ds["mesh2d_nFaces"].values.shape[0]), dtype=float)
coastal_gdf.set_crs(32618, inplace=True)

In [ ]:
cell_areas = coastal_gdf.area.values
cell_areas

In [ ]:
time_str = results_ds["time"].values
time_str, time_str[-1]

In [ ]:
output_freq = int(np.timedelta64(1, "D") / (time_str[1] - time_str[0]))
time_index = np.arange(0, time_str.shape[0], output_freq)
time_index.shape, time_index[1:]

In [ ]:
time_mf = time_str[time_index]
print(time_mf.shape, time_mf)

In [ ]:
gp_gpd = gpd.read_file("../modflow/gis/greenpoint_onshore_offshore_utm18n.shp")

#### Load the MODFLOW model

In [ ]:
mf_run_path = pl.Path(f"../modflow/greenport500ft/run_{mf_tag}/")
mf_npz_path = mf_run_path

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=mf_run_path)
gwf = sim.get_model()

In [ ]:
hobj = gwf.output.head()

In [ ]:
totimes = hobj.get_times()
print(f"{len(totimes)}\n{totimes}")

In [ ]:
bobj = gwf.output.budget()

In [ ]:
bobj.get_unique_record_names()

#### Calculate the mean recharge

In [ ]:
mean_recharge = []
for totim in totimes:
    rQ = bobj.get_data(text="RCH", totim=totim)[0]
    q = rQ["q"]
    idx = q > 0.
    mean_recharge.append(q[idx].mean())
    
mean_recharge = np.array(mean_recharge) * (conversion_factor / (500. * 500.))

In [ ]:
mean_recharge.shape, mean_recharge.max()

In [ ]:
df = pd.DataFrame(data=mean_recharge)
df.set_index(time_mf[:mean_recharge.shape[0]],inplace=True)

In [ ]:
df

#### Load ghb shapefile

In [ ]:
mf6_gpd = gpd.read_file("../modflow/gis/greenpoint_ghb_4456.shp")
mf6_gpd["head_difference"] = np.zeros((mf6_gpd.shape[0]), dtype=float)
mf6_gpd

In [ ]:
mf6_gpd.crs

#### Load active grid shapefile

In [ ]:
mf6_grid_gpd = gpd.read_file("../modflow/gis/greenpoint_onshore_offshore_4456.shp")
mf6_grid_gpd

#### Set the crs for the MODFLOW model

In [ ]:
gwf.modelgrid.crs = mf6_gpd.crs

#### Load ghb data calculated using coastal model results

In [ ]:
ghb_elev = np.load(f"{mf_npz_path}/ghb_elev.npz")
ghb_elev

In [ ]:
ghb_cond = np.load(f"{mf_npz_path}/ghb_cond.npz")
ghb_cond

In [ ]:
qext_elev = np.load(f"{mf_npz_path}/qext.npz")
qext_elev

In [ ]:
qsize, qmin, qmax = 0, 1e30, -1e30
for key, value in qext_elev.items():
    qsize = value.shape[0]
    qmin = min(qmin, np.nanmin(value))
    qmax = max(qmax, np.nanmax(value))
qmin *= 3.28081 * 12.0
qmax *= 3.28081 * 12.0
qsize, qmin, qmax

In [ ]:
qmin, qmax = -8.0, 8.0

In [ ]:
qvalue = np.full(qsize, np.nan, dtype=float)

In [ ]:
time_keys = ghb_elev.files
sampled_time_keys = time_keys[mf_output_sample-1::mf_output_sample]
print(
    len(time_keys),
    len(sampled_time_keys), 
    sampled_time_keys
)

In [ ]:
v = hobj.get_data(totim=1.)[0]
v[v == 1.e30] = 0.
v.min(), v.max(), v.mean()

In [ ]:
gwf.modelgrid.crs

In [ ]:
totim = 1.0
coastal_idx = int(totim) + 1
title_str = set_title_string(time_str[coastal_idx])  
mm = flopy.plot.PlotMapView(model=gwf)
# hc = mm.plot_array(hobj.get_data(totim=1.0))
hc = mm.plot_array(bobj.get_data(text="GHB", totim=totim, full3D=True)[0])
mf6_gpd.plot(ax=mm.ax, lw=0.5, ec="black", color="none")
plt.colorbar(hc, ax=mm.ax, shrink=0.75)
cx.add_basemap(mm.ax, crs=mf6_gpd.crs, **basemap_kwds)
mm.ax.set_title(f"GHB flux at {title_str}", size=8);

In [ ]:
mosaic_list = [
    ["c","c","c","c"],
    ["a","a","b","b"],
    ["a","a","b","b"],
    ["a","a","b","b"],
    ["a","a","b","b"],
    ]

In [ ]:
if units == "in":
    head_levels = [1, 2, 3]
    head_min, head_max = -1, 5
    stage_min, stage_max = -3, 3
    rch_min, rch_max = 0, 3
    flux_min, flux_max = -1, 1
elif units == "mm":
    head_levels = [0.25, 0.5, 0.75, 1.0]
    head_min, head_max = -0.25, 2
    stage_min, stage_max = -1, 1
    rch_min, rch_max = 0, 100
    flux_min, flux_max = -25, 25


#### MODFLOW 6 results

In [ ]:
totim = 50
npz_idx = sampled_time_keys[totim]
with styles.USGSMap():
    fig, axd = plt.subplot_mosaic(
    mosaic_list,
    layout="constrained", 
    figsize=figsize,
    )    

    ax = axd["c"]
    ax.set_xlim(-0.5,mean_recharge.shape[0]-.5)
    ax.set_ylim(rch_min, rch_max)

    df.plot(kind="bar", ax=ax, legend=False, zorder=111, color="blue")
    fb = ax.fill_between([-0.5, totim+0.5], rch_max, **progress_kwds)

    ax.xaxis.set_major_locator(MonthLocator())
    ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=16))
    
    ax.xaxis.set_major_formatter(ticker.NullFormatter())
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%B'))
    
    # Remove the tick lines
    ax.tick_params(axis='x', which='minor', tick1On=False, tick2On=False)
    
    # Align the minor tick label
    for label in ax.get_xticklabels(minor=True):
        label.set_horizontalalignment('center')

    ax.set_ylabel("Recharge")
    styles.add_text(ax, text=f"{units} / day", x=0.99, y=0.95, bold=False, va="top", ha="right")
    
    title_str = set_title_string(time_mf[int(totim)])  
    fig.suptitle(title_str, fontsize=8)

    h = hobj.get_data(totim=totim)
    h[h < 1e29] *= head_conversion_factor
    q = bobj.get_data(text="GHB", totim=totim, full3D=True)[0]
    q[q == 0.0] = np.nan   
    q *= conversion_factor / (500. * 500.)
    
    ax = axd["a"]
    mm0 = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    hc = mm0.plot_array(h, alpha=alpha, vmin=head_min, vmax=head_max)
    mf6_grid_gpd.plot(ax=ax, lw=0.25, ec="black", color="none")
    ca = mm0.contour_array(h, levels=head_levels, colors="white", linewidths=0.5)
    plt.colorbar(hc, ax=ax, shrink=0.5, orientation="horizontal")
    cx.add_basemap(ax, crs=mf6_gpd.crs, **basemap_kwds)
    
    ax.set_title(f"Head ({head_units})", size=8)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    

    ax = axd["b"]
    mm1 = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    ghbq = mm1.plot_array(-q, alpha=alpha, cmap="RdBu", vmin=flux_min , vmax=flux_max)
    mf6_gpd.plot(ax=ax, lw=0.25, ec="black", color="none")
    plt.colorbar(ghbq, ax=ax, shrink=0.5, extend="both", orientation="horizontal")
    cx.add_basemap(ax, crs=mf6_gpd.crs, **basemap_kwds)
    
    ax.set_title(f"Coastal discharge ({units}/day)", size=8)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # coastal model inset
    ax_ins = axd["b"].inset_axes(
        [-0.125, 0.55, 0.45, 0.45],
        xticklabels=[], 
        yticklabels=[],
    )
    wdp = full_results_ds["mesh2d_s1"].isel(time=time_index[totim]).ugrid.plot(animated=True, ax=ax_ins, cmap="RdBu", vmin=flux_min, vmax=flux_max, lw=0.0, alpha=alpha, add_colorbar=False)
    gdfp = coastal_gdf.plot(color="black", fc="none", lw=0.25, ax=ax_ins)

    qvalue = qext_elev[str(npz_idx)]
    qvalue *= flux_conversion_factor * 86400. / cell_areas
    qvalue[qvalue == 0.0] = np.nan
    wdp.set_array(qvalue)

    ax_ins.set_xlim(extent[:2])
    ax_ins.set_ylim(extent[2:])
    cx.add_basemap(ax_ins, crs=gp_gpd.crs, attribution=False, source=cx_provider, zoom=11)

    ax_ins.set_title(f"Coastal Exchange ({flux_units})", size=8)
    ax_ins.set_xlabel(None)
    ax_ins.set_ylabel(None)
    ax_ins.set_xticklabels([])
    ax_ins.set_yticklabels([])      

    fig.align_labels() 

    fig.savefig(f"figures/greenport_modflow_dflow_results{fig_ext}", dpi=300, transparent=transparent)

#### Animate MODFLOW Results

In [ ]:
hr_sample = 24
days_per_sec = 2
fps = int(days_per_sec * 24 / hr_sample)
fps

In [ ]:
time_index[1:].shape[0], len(totimes)

In [ ]:
frame1 = min(len(totimes), time_index[1:].shape[0])
frames = np.arange(0, frame1, 1)[1:]
frame1, frames.shape

In [ ]:
ani_ext = ".mp4"
Writer = mpl.animation.writers["ffmpeg"]
writer = Writer(fps=fps, metadata=dict(artist="jdhughes"), bitrate=2056)

In [ ]:
with styles.USGSMap():
    fig, axd = plt.subplot_mosaic(
    mosaic_list,
    layout="constrained", 
    figsize=figsize,
    )    

    ax = axd["c"]
    ax.set_xlim(-0.5, mean_recharge.shape[0]-.5)
    ax.set_ylim(0, rch_max)

    df.plot(kind="bar", ax=ax, legend=False, zorder=111, color="blue")
    fb = ax.fill_between([-0.5, -0.5], rch_max, **progress_kwds)

    ax.xaxis.set_major_locator(MonthLocator())
    ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=16))
    
    ax.xaxis.set_major_formatter(ticker.NullFormatter())
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%B'))
    
    # Remove the tick lines
    ax.tick_params(axis='x', which='minor', tick1On=False, tick2On=False)
    
    # Align the minor tick label
    for label in ax.get_xticklabels(minor=True):
        label.set_horizontalalignment('center')

    ax.set_ylabel("Recharge")
    styles.add_text(ax, text=f"{units} / day", x=0.99, y=0.95, bold=False, va="top", ha="right")    
    
    title_str = set_title_string(time_mf[0]) 
    fig.suptitle(title_str, fontsize=8)

    h = hobj.get_data(totim=totim)
    h[h < 1e29] *= head_conversion_factor
    q = bobj.get_data(text="GHB", totim=totim, full3D=True)[0]
    q[q == 0.0] = np.nan
    q *= conversion_factor / (500. * 500.)
    
    ax = axd["a"]
    
    mm0 = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    hc = mm0.plot_array(h, alpha=alpha, vmin=head_min, vmax=head_max)
    mf6_grid_gpd.plot(ax=ax, lw=0.25, ec="black", color="none")
    C = mm0.contour_array(h, levels=head_levels, colors="white", linewidths=0.5)
    plt.colorbar(hc, ax=ax, shrink=0.5, orientation="horizontal")
    cx.add_basemap(ax, crs=mf6_gpd.crs, **basemap_kwds)
    
    ax.set_title(f"Head ({head_units})", size=8)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    ax = axd["b"]
    mm1 = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    ghbq = mm1.plot_array(-q, alpha=alpha, cmap="RdBu", vmin=flux_min, vmax=flux_max)
    mf6_gpd.plot(ax=ax, lw=0.25, ec="black", color="none")
    plt.colorbar(ghbq, ax=ax, shrink=0.5, extend="both", orientation="horizontal")
    cx.add_basemap(ax, crs=mf6_gpd.crs, **basemap_kwds)

    ax.set_title(f"Coastal discharge ({units}/day)", size=8)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # coastal model inset
    ax_ins = axd["b"].inset_axes(
        [-0.125, 0.55, 0.45, 0.45],
        xticklabels=[], 
        yticklabels=[],
    )
    wdp = full_results_ds["mesh2d_s1"].isel(time=time_index[totim]).ugrid.plot(animated=True, ax=ax_ins, cmap="RdBu", vmin=flux_min, vmax=flux_max, lw=0.0, alpha=alpha, add_colorbar=False)
    gdfp = coastal_gdf.plot(color="black", fc="none", lw=0.25, ax=ax_ins)
    wdp.set_array(qvalue)

    ax_ins.set_xlim(extent[:2])
    ax_ins.set_ylim(extent[2:])
    cx.add_basemap(ax_ins, crs=gp_gpd.crs, attribution=False, source=cx_provider, zoom=11)

    ax_ins.set_title(f"Coastal Exchange ({units})", size=8)
    ax_ins.set_xlabel(None)
    ax_ins.set_ylabel(None)
    ax_ins.set_xticklabels([])
    ax_ins.set_yticklabels([])    
     

    fig.align_labels()    
    
    def func(idx):
        global C, fb

        ax = axd["c"]
        fb.remove()
        fb = ax.fill_between([-0.5, float(idx)+0.5], rch_max, **progress_kwds)
        
        on_time = set_title_string(time_mf[idx])
        fig.suptitle(on_time, fontsize=8)

        totim = totimes[idx]

        h = hobj.get_data(totim=totim)
        h[h < 1e29] *= head_conversion_factor
        q = bobj.get_data(text="GHB", totim=totim, full3D=True)[0]

        h[h == 1e30] = np.nan
        q[q == 0.0] = np.nan
        q *= conversion_factor / (500. * 500.)
        
        ax = axd["a"]
        hc.set_array(h[0])
        C.remove()
        C = mm0.contour_array(h, levels=head_levels, colors="white", linewidths=0.5)
        
        ax = axd["b"]
        ghbq.set_array(-q[0])

        qvalue = qext_elev[str(sampled_time_keys[idx])]
        qvalue *= flux_conversion_factor * 86400./ cell_areas
        qvalue[qvalue == 0.0] = np.nan
        # print(np.nanmin(qvalue), np.nanmax(qvalue))
        wdp.set_array(qvalue)
        
        return C, fb, wdp
    
    ani = FuncAnimation(fig, func, frames=frames, blit=False)
    
    plt.close()

ani.save(animation_ws / f"greenport_modflow_dflow_results{ani_ext}", writer=writer)
# HTML(ani.to_jshtml())

In [ ]:
head_idx = [(l, r, c) for l, r, c in zip(mf6_gpd["layer"], mf6_gpd["row"], mf6_gpd["column"])]
head_idx[:10]

In [ ]:
# specific to dflow
stage = results_ds["mesh2d_s1"]
depth = results_ds["mesh2d_waterdepth"]

In [ ]:
# specific to dflow
def mask_stage(idx):
    v = stage.isel(time=time_index[idx]).values
    d = depth.isel(time=time_index[idx]).values
    v[d < 0.001] = np.nan
    return v * stage_conversion_factor
    

In [ ]:
totim = 50
npz_idx = sampled_time_keys[totim]
with styles.USGSMap():
    fig, axd = plt.subplot_mosaic(
    mosaic_list,
    layout="constrained", 
    figsize=figsize,
    )    

    ax = axd["c"]
    ax.set_xlim(-0.5,mean_recharge.shape[0]-.5)
    ax.set_ylim(rch_min, rch_max)

    df.plot(kind="bar", ax=ax, legend=False, zorder=111, color="blue")
    fb = ax.fill_between([-0.5, totim+0.5], rch_max, **progress_kwds)

    ax.xaxis.set_major_locator(MonthLocator())
    ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=16))
    
    ax.xaxis.set_major_formatter(ticker.NullFormatter())
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%B'))
    
    # Remove the tick lines
    ax.tick_params(axis='x', which='minor', tick1On=False, tick2On=False)
    
    # Align the minor tick label
    for label in ax.get_xticklabels(minor=True):
        label.set_horizontalalignment('center')

    ax.set_ylabel("Recharge")
    styles.add_text(ax, text=f"{units} / day", x=0.99, y=0.95, bold=False, va="top", ha="right")
    
    title_str = set_title_string(time_mf[int(totim)])
    fig.suptitle(title_str, fontsize=8)

    bh = ghb_elev[str(npz_idx)] * head_conversion_factor
    h = hobj.get_ts(head_idx)[int(totim), 1:] * head_conversion_factor
    hd = h - bh
    
    cnd = ghb_cond[str(npz_idx)]
    mask = cnd == 0.0
    
    bh[mask] = np.nan
    hd[mask] = np.nan
    cnd[mask] = np.nan
    
    ax = axd["a"]
    ax.set_xlim(gwf.modelgrid.extent[0:2])
    ax.set_ylim(gwf.modelgrid.extent[2:])
    mf6_gpd.plot(ax=ax, alpha=alpha, column="bhead", lw=0.25, ec="black", vmin=stage_min, vmax=stage_max)
    v = ax.collections[0]
    v.set_array(bh)
    plt.colorbar(v, ax=ax, shrink=0.5, extend="both", orientation="horizontal")
    cx.add_basemap(ax, crs=mf6_gpd.crs, **basemap_kwds)

    ax.set_title(f"Stage ({stage_units})", size=8)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # coastal model inset
    ax_ins = axd["a"].inset_axes(
        [-0.125, 0.55, 0.45, 0.45],
        xticklabels=[], 
        yticklabels=[],
    )
    wdp = stage.isel(time=time_index[totim]).ugrid.plot(animated=True, ax=ax_ins, vmin=stage_min, vmax=stage_max, lw=0.0, alpha=0.25, add_colorbar=False)
    gdfp = coastal_gdf.plot(color="black", fc="none", lw=0.25, ax=ax_ins)
    wdp.set_array(mask_stage(totim))

    ax_ins.set_xlim(extent[:2])
    ax_ins.set_ylim(extent[2:])
    cx.add_basemap(ax_ins, crs=gp_gpd.crs, attribution=False, source=cx_provider, zoom=11)

    ax_ins.set_title(f"DFLOW Stage ({stage_units})", size=8)
    ax_ins.set_xlabel(None)
    ax_ins.set_ylabel(None)
    ax_ins.set_xticklabels([])
    ax_ins.set_yticklabels([])    

    ax = axd["b"]
    ax.set_xlim(gwf.modelgrid.extent[0:2])
    ax.set_ylim(gwf.modelgrid.extent[2:])
    # mf6_gpd.plot(ax=ax, alpha=alpha, column="cond", lw=0.25, ec="black")
    mf6_gpd.plot(ax=ax, alpha=alpha, column="head_difference", lw=0.25, ec="black")
    v = ax.collections[0]
    # v.set_array(cnd)
    v.set_array(hd)
    plt.colorbar(v, ax=ax, shrink=0.5, orientation="horizontal")
    cx.add_basemap(ax, crs=mf6_gpd.crs, **basemap_kwds)
    
    ax.set_title(f"Head difference ({head_units})", size=8)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    fig.align_labels()    

    fig.savefig(f"figures/greenport_modflow_dflow_ghb{fig_ext}", dpi=300, transparent=transparent)

#### Animate GHB data

In [ ]:
with styles.USGSMap():
    fig, axd = plt.subplot_mosaic(
        mosaic_list,
        layout="constrained",
        figsize=figsize,
    )    

    ax = axd["c"]
    ax.set_xlim(-0.5,mean_recharge.shape[0]-.5)
    ax.set_ylim(rch_min, rch_max)

    df.plot(kind="bar", ax=ax, legend=False, zorder=111, color="blue")
    fb = ax.fill_between([-0.5, -0.5], rch_max, **progress_kwds)

    ax.xaxis.set_major_locator(MonthLocator())
    ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=16))
    
    ax.xaxis.set_major_formatter(ticker.NullFormatter())
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%B'))
    
    # Remove the tick lines
    ax.tick_params(axis='x', which='minor', tick1On=False, tick2On=False)
    
    # Align the minor tick label
    for label in ax.get_xticklabels(minor=True):
        label.set_horizontalalignment('center')

    ax.set_ylabel("Recharge")
    styles.add_text(ax, text=f"{units} / day", x=0.99, y=0.95, bold=False, va="top", ha="right")    

    title_str = set_title_string(time_mf[0])
    fig.suptitle(title_str, fontsize=8)

    ax = axd["a"]
    ax.set_xlim(gwf.modelgrid.extent[0:2])
    ax.set_ylim(gwf.modelgrid.extent[2:])
    mf6_gpd.plot(ax=ax, alpha=alpha, column="bhead", lw=0.5, ec="black", vmin=stage_min, vmax=stage_max)
    plt.colorbar(ax.collections[0], ax=ax, shrink=0.5, extend="both", orientation="horizontal")
    cx.add_basemap(ax, crs=mf6_gpd.crs, **basemap_kwds)
    
    ax.set_title(f"Coastal stage ({stage_units})", size=8)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    ax = axd["b"]
    ax.set_xlim(gwf.modelgrid.extent[0:2])
    ax.set_ylim(gwf.modelgrid.extent[2:])
    mf6_gpd.plot(ax=ax, alpha=alpha, column="head_difference", lw=0.25, ec="black")
    plt.colorbar(ax.collections[0], ax=ax, shrink=0.5, orientation="horizontal")
    cx.add_basemap(ax, crs=mf6_gpd.crs, **basemap_kwds)
    
    ax.set_title(f"Head difference ({stage_units})", size=8)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # coastal model inset
    ax_ins = axd["a"].inset_axes(
        [-0.125, 0.55, 0.45, 0.45],
        xticklabels=[], yticklabels=[],
    )
    wdp = stage.isel(time=time_index[0]).ugrid.plot(animated=True, ax=ax_ins, vmin=stage_min, vmax=stage_max, lw=0.0, alpha=0.25, add_colorbar=False)
    gdfp = coastal_gdf.plot(color="black", fc="none", lw=0.25, ax=ax_ins)
    wdp.set_array(mask_stage(0))
    
    ax_ins.set_title(f"DFLOW Stage ({stage_units})", size=8)
    ax_ins.set_xlim(extent[0:2])
    ax_ins.set_ylim(extent[2:])
    ax_ins.set_xlabel(None)
    ax_ins.set_ylabel(None)

    cx.add_basemap(ax_ins, crs=gp_gpd.crs, attribution=False, source=cx_provider, zoom=11)

    fig.align_labels()
    
    def func(idx):
        global fb, wdp
        
        ax = axd["c"]
        fb.remove()
        fb = ax.fill_between([-0.5, float(idx)+0.5], rch_max, **progress_kwds)
        
        on_time = set_title_string(time_mf[idx])
        fig.suptitle(on_time, fontsize=8)
        
        bh = ghb_elev[str(sampled_time_keys[idx])] * head_conversion_factor
        h = hobj.get_ts(head_idx)[idx, 1:] * head_conversion_factor
        hd = h - bh
        
        cnd = ghb_cond[str(sampled_time_keys[idx])]
        idx_map = (cnd == 0.)
        
        bh[idx_map] = np.nan
        hd[idx_map] = np.nan
        
        ax = axd["a"]
        ax.collections[0].set_array(bh)
        
        ax = axd["b"]
        ax.collections[0].set_array(hd)

        wdp.set_array(mask_stage(idx))
        
        return fb, wdp

    ani = FuncAnimation(fig, func, frames=frames, blit=False)
    
    plt.close()

ani.save(animation_ws / f"greenport_modflow_dflow_ghb{ani_ext}", writer=writer)
# HTML(ani.to_jshtml())